In [1]:
import pandas as pd
import numpy as np
import pyarrow.feather as feather
import warnings
warnings.filterwarnings('ignore')

In [2]:
target_df_train = pd.read_csv('/Users/dmitry/Downloads/likes_data/train', header=None, sep='\t')
target_df_test = pd.read_csv('/Users/dmitry/Downloads/likes_data/test.txt', header=None, sep='\t')

In [3]:
def make_dataset(data: pd.DataFrame) -> pd.DataFrame:
    """
    Making long dataset
    :params data: your data frame
    :return: pd.DataFrame
    """
    data[0] = [list(row.split())[::-1] for row in data[0]]
    data = data.explode(0).reset_index().rename(columns={'index': 'user_id', 0: 'items_id'})
    data['order'] = data.groupby(data['user_id']).cumcount()
    
    return data

In [4]:
target_df_train[:3]

,0
0,333396 267089 155959 353335 414000 339989 2741...
1,174197 335779 141676 119856 376664 311755 3881...
2,254861 213397 462645 403619 446211 221833 1572...


In [5]:
target_df_test[:3]

,0
0,454758 382341 240893 280388 362253 436737 2576...
1,294661 374689 182272 102025 70927 86844 280488...
2,15296 384304 138151 474141 182935 262020 21145...


In [6]:
target_df_train = make_dataset(target_df_train)
target_df_test = make_dataset(target_df_test)

In [41]:
target_df_train[:10]

,user_id,items_id,order
0,0,388242,0
1,0,278503,1
2,0,102795,2
3,0,470957,3
4,0,159637,4
5,0,95202,5
6,0,196872,6
7,0,352770,7
8,0,319811,8
9,0,122447,9


In [8]:
target_df_test[:10]

,user_id,items_id,order
0,0,471705,0
1,0,219426,1
2,0,101168,2
3,0,361110,3
4,0,22932,4
5,0,291870,5
6,0,404030,6
7,0,127295,7
8,0,162625,8
9,0,200945,9


In [10]:
feather.write_feather(target_df_train, '/Users/dmitry/Library/CloudStorage/GoogleDrive-ceo@gangai.pro/Мой диск/Проекты/recsys/HW/data/processed/train_processed.feather')
feather.write_feather(target_df_test, '/Users/dmitry/Library/CloudStorage/GoogleDrive-ceo@gangai.pro/Мой диск/Проекты/recsys/HW/data/processed/test_processed.feather')

In [12]:
target_df_train.to_hdf(
    '/Users/dmitry/Library/CloudStorage/GoogleDrive-ceo@gangai.pro/Мой диск/Проекты/recsys/HW/data/processed/train_processed.h5', key='df')
target_df_test.to_hdf(
    '/Users/dmitry/Library/CloudStorage/GoogleDrive-ceo@gangai.pro/Мой диск/Проекты/recsys/HW/data/processed/test_processed.h5', key='df')

In [37]:
target_df_test = feather.read_feather('/Users/dmitry/Library/CloudStorage/GoogleDrive-ceo@gangai.pro/Мой диск/Проекты/recsys/HW/data/processed/test_processed.feather')

In [42]:
target_df_test['user_id'] = target_df_test['user_id'] + target_df_train['user_id'].nunique()

In [43]:
target_df_test

,user_id,items_id,order
0,1160084,471705,0
1,1160084,219426,1
2,1160084,101168,2
3,1160084,361110,3
4,1160084,22932,4
...,...,...,...
23262195,1449997,169689,34
23262196,1449997,215394,35
23262197,1449997,29495,36
23262198,1449997,410120,37


In [50]:
target_df = pd.concat([target_df_train, target_df_test], ignore_index=True)

In [51]:
target_df

,user_id,items_id,order
0,0,388242,0
1,0,278503,1
2,0,102795,2
3,0,470957,3
4,0,159637,4
...,...,...,...
117450829,1449997,169689,34
117450830,1449997,215394,35
117450831,1449997,29495,36
117450832,1449997,410120,37


In [52]:
feather.write_feather(target_df, '/Users/dmitry/Library/CloudStorage/GoogleDrive-ceo@gangai.pro/Мой диск/Проекты/recsys/HW/data/processed/target_df.feather')
target_df.to_hdf(
    '/Users/dmitry/Library/CloudStorage/GoogleDrive-ceo@gangai.pro/Мой диск/Проекты/recsys/HW/data/processed/target_df.h5', key='df')